## Imports
Imports some library to using a collection of functions that can be added to my project in python code and called as necessary.

In [1]:
import numpy as np
import cv2 as cv
import os
import datetime
import winsound
from tkinter import*
from keras.models import load_model

## Haar Cascade Algorithm
The haar cascade algorithm recognize or detect the face to find human’s eyes and used to classify objects based on their interesting features.

In [2]:
face_cascade = cv.CascadeClassifier('Haar Cascade Files\haarcascade_frontalface_alt.xml') ## Front Face
right_eye_cascade = cv.CascadeClassifier('Haar Cascade Files\haarcascade_righteye_2splits.xml') ## Right Eye
left_eye_cascade = cv.CascadeClassifier('Haar Cascade Files\haarcascade_lefteye_2splits.xml') ## Left Eye

## Loading Model (CNN)

In [3]:
model = load_model('Model/Drowsiness Driver Detection_Model.h5') ## Loading Model to predict whether eye are open or closed

## Implementation
Collect a dataset from a reliable source and then send it to be preprocessing which filter images and sends them to a convolutional neural network (CNN) to predict whether eyes closed or open. The trained dataset is made up of multiple images with open and closed eyes. When using the haar cascade classifier to classify an object and extracting a face expression and left and right eye from XML files to be detected. The only data for left eye feed it into CNN classifier to predict the results will be presented as if an eye is closed or open. The same process is used with the right eye data. CNN will predict whether eyes are closed or open.

In [4]:
def startDetection(): ## Start Function it include detection implementation
    path = os.getcwd()
    capture = cv.VideoCapture(0)
    font = cv.FONT_HERSHEY_COMPLEX_SMALL
    
    label = ['Open' , 'Closed'] ## label array for type of string include Open or Closed elements
    right_predict = [[99,99]]
    left_predict = [[99,99]]
    
    detect = "" 
    score = 0
    frquency = 2000
    duration = 500
    
    while True:
        ret, frame = capture.read()
        height,width = frame.shape[:2]
        input_RGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        
        faces = face_cascade.detectMultiScale(input_RGB,minNeighbors=5,scaleFactor=1.3,minSize=(25,25))
        right_eye = right_eye_cascade.detectMultiScale(input_RGB)
        left_eye = left_eye_cascade.detectMultiScale(input_RGB)
        
        if faces == ():
            cv.putText(frame, 'Face Not Found', (250,height-20), font, 1, (255,255,255), 2, cv.LINE_AA)
            score = 0
        else:
            for (x,y,w,h) in faces:
                if(detect == 'Open'):
                    cv.rectangle(frame, (x,y) , (x+w,y+h) , (0,255,0) , 2)
                else:
                    cv.rectangle(frame, (x,y) , (x+w,y+h) , (0,0,255) , 2)
                
            for (x,y,w,h) in right_eye:
                r_eye  = frame[y:y+h,x:x+w]
                r_eye = cv.cvtColor(r_eye,cv.COLOR_BGR2RGB)
                r_eye = cv.resize(r_eye,(24,24))
                r_eye = r_eye/255
                r_eye =  r_eye.reshape(24,24,3)
                r_eye = np.expand_dims(r_eye,axis=0)
                right_predict = model.predict(r_eye)
                #cv.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 2)
                                
                if(right_predict[0][0] < right_predict[0][1]):
                    label = 'Open'
                elif(right_predict[0][0] > right_predict[0][1]):
                    label = 'Closed'
                break
                        
            for (x,y,w,h) in left_eye:
                l_eye = frame[y:y+h,x:x+w]
                l_eye = cv.cvtColor(l_eye,cv.COLOR_BGR2RGB)
                l_eye = cv.resize(l_eye,(24,24))
                l_eye = l_eye/255
                l_eye = l_eye.reshape(24,24,3)
                l_eye = np.expand_dims(l_eye,axis=0)
                left_predict = model.predict(l_eye)
                #cv.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 2)
                
                if(left_predict[0][0] < left_predict[0][1]):
                    label = 'Open'
                elif(left_predict[0][0] > left_predict[0][1]):
                    label = 'Closed'
                break
                    
            if((right_predict[0][0] > right_predict[0][1]) and (left_predict[0][0] > left_predict[0][1])):
                display = cv.putText(frame, "Closed", (10,height-20), font, 1, (0,0,255), 2, cv.LINE_AA)
                score = score + 1
                detect = 'Closed'
            else:
                display = cv.putText(frame, "Open", (10,height-20), font, 1, (0,255,0), 2, cv.LINE_AA)
                score = score - 1
                detect = 'Open'
                
            if(score < 0):
                score = 0
            display = cv.putText(frame, "Score: "+str(score), (100,height-20), font, 1, (255,255,255), 2, cv.LINE_AA)
            
            if(score > 10):
                try:
                    winsound.Beep(frquency,duration)
                    if(detect == 'Open'):
                        score = 9
                except:
                    pass
             
        cv.imshow('Drowsiness Driver Detection Webcam',frame)  
        key=cv.waitKey(1)
        if key%256 == 27:
            break
    capture.release()
    cv.destroyAllWindows()

## GUI
Control the system from user interface like start detection, exit system, close detection.

In [5]:
gui = Tk()
gui.title('Drowsiness Driver Detection System Alert')

sc_width = 1000
sc_height = 650

date = datetime.datetime.now()

img = PhotoImage(file = "Cover GUI/Cover Image.png")
Label(gui,image=img,bg='white').place(x=25 , y=80)

screen_width =  gui.winfo_screenwidth()
screen_height = gui.winfo_screenheight()

x_center = (screen_width / 2) - (sc_width / 2)
y_center = (screen_height / 2) - (sc_height/2)

gui.geometry(f'{sc_width}x{sc_height}+{int(x_center)}+{int(y_center)}')
gui.configure(bg = 'white')

frame = Frame(gui, width = 500, height = 350, bg = 'white')
frame.place(relx = 0.7, rely = 0.2, anchor = CENTER)

heading_Title = Label(frame, text = 'Drowsiness Driver Detection System', fg = 'black', bg = 'white',font = ('Microsoft YaHei UI Light', 20, 'bold'))
heading_Title.place(relx = 0.5, rely = 0.5, anchor = CENTER)

dateTime = Label(frame,text=f"{date:%A, %B %d, %Y}",fg = 'gray', bg = 'white', font = ('Calibri', 12, 'bold'))
dateTime.place(relx = 0.5, rely=0.35, anchor = CENTER)

Start_Button = Button(frame,width = 20, pady = 6, text = 'Start Detection',  fg = 'white', bg = 'black',border = 0, command = startDetection)
Start_Button.place(relx = 0.5, rely = 0.8, anchor = CENTER)

ExitWindow_Button = Button(frame,width = 20, pady = 6, text = 'Exit Window', fg = 'white', bg = 'black',border = 0, command = gui.destroy)
ExitWindow_Button.place(relx = 0.5, rely = 0.95, anchor = CENTER)

gui.mainloop()

<ipython-input-4-cd8f4b8c00e7>:24: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if faces == ():
